## **Q1)** 

현재 threshold가 0.3으로 매우 낮아 acceptance rate가 매우 높다.
그러므로 FP가 높고 FN가 낮다.

FN은 Type 1 Error와 같은 개념이며, FP는 Type 2 Error와 같은 개념이다.

그러므로 이 경우에서는 Type 2 Error가 높다.

## **Q2-1)**

Accuracy : 전체 중 제대로 맞춘 비율

Accuracy = (TP +TN) / (TP + FN + FP+ TN)


Precision: True라고 판단한 것 중 실제로 True인 것의 비율

Precision = TP / (TP + FP)

Recall: 실제로 True일 때, True라고 판단한 것의 비율

Recall = TP / (TP + FN)

## **Q2-2)** 

날씨를 예측하는 시스템

내일 날씨가 맑을 지 아니면 비가 내릴 지를 예측하는 시스템이 있다고 가정한다.  


그리고 비가 온다는 것을 True라고 하자.  


이때 이 시스템을 어느 정도 믿을 것인지에 대한 것이 threshold가 될 것이다.

이 때 True가 될 확률의 threshold를 높인다면 날씨 예측 시스템의 결과를 웬만해선 잘 믿지 않는 것이다.  

즉, 시스템에서 비가 온다 하여도 이를 믿지 않는 것이다.

이 예시에서는 True가 될 확률의 threshold를 낮추는 것이 더 합리적일 것이라 생각한다.

비가 온다는 시스템의 결과를 믿지 않았는데 비가 오면 우산을 사야하는 비용적 손해가 있지만,  

비가 온다는 시스템을 믿었는데 비가 오지 않을 때는 이러한 비용적 손해는 존재하지 않기 때문이다.  

(짐이 늘어난다는 거 빼곤 손해볼 게 없을 것이다.)


## **Q3)**

In [10]:
#torch 불러오기

import torch 
import torch.nn as nn 
import torchvision.datasets as dsets 
import torchvision.transforms as transforms 
from torch.autograd import Variable 

# MNIST 데이터셋 불러오기 (train과 test 데이터셋으로 나누어 부름)
train_dataset = dsets.MNIST(root ='./data',  
                            train = True,  
                            transform = transforms.ToTensor(), 
                            download = True) 
  
test_dataset = dsets.MNIST(root ='./data',  
                           train = False,  
                           transform = transforms.ToTensor()) 


#기본적인 parameter 값 설정
input_size = 784
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001
  
# 데이터셋을 불러올 loader 설정
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,batch_size = batch_size, shuffle = True) 
  
test_loader = torch.utils.data.DataLoader(dataset = test_dataset, batch_size = batch_size,shuffle = False) 



#로지스틱 회귀와 forward pass 함수를 정의
class LogisticRegression(nn.Module): 
    def __init__(self, input_size, num_classes): 
        super(LogisticRegression, self).__init__() 
        self.linear = nn.Linear(input_size, num_classes) 
  
    def forward(self, x): 
        out = self.linear(x) 
        return out 
    
#모델을 정의
model = LogisticRegression(input_size, num_classes)

#Loss function을 crossentropyloss로 정의함
criterion = nn.CrossEntropyLoss() 

#optimiser를 SGD로 정의하고, learning rate를 설정, 해당 값은 위의 parameter에서 설정하였음

optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate) 


# 모델을 train하는 코드 
for epoch in range(num_epochs): 
    for i, (images, labels) in enumerate(train_loader): 
        images = Variable(images.view(-1, 28 * 28))  #데이터셋의 크기를 1*784로 변환
        labels = Variable(labels) 
  
        
        optimizer.zero_grad() #gradient를 0으로 초기화 
        outputs = model(images) 
        loss = criterion(outputs, labels) #loss 값을 설정 
        loss.backward() #back propagation을 수행
        optimizer.step() #SGD를 활용한 optimization을 수행
  

        #epoch과 step의 진행상황 그리고 Loss의 값을 표시하는 line
        if (i + 1) % 100 == 0: 
            print('Epoch: [% d/% d], Step: [% d/% d], Loss: %.4f'% (epoch + 1, num_epochs, i + 1, len(train_dataset) // batch_size, loss.data))
            
            
#train set을 통해 학습한 모델을 test set을 통해 정확도를 구함
correct = 0
total = 0
for images, labels in test_loader: 
    images = Variable(images.view(-1, 28 * 28)) 
    outputs = model(images) 
    _, predicted = torch.max(outputs.data, 1) 
    total += labels.size(0) 
    correct += (predicted == labels).sum() 

#test set에 대한 모델의 정확도를 출력하는 line    
print ('Accuracy of the model on the 10000 test images: % d %%' % (100 * correct / total)) 



Epoch: [ 1/ 5], Step: [ 100/ 600], Loss: 2.2164
Epoch: [ 1/ 5], Step: [ 200/ 600], Loss: 2.1189
Epoch: [ 1/ 5], Step: [ 300/ 600], Loss: 2.0428
Epoch: [ 1/ 5], Step: [ 400/ 600], Loss: 1.9594
Epoch: [ 1/ 5], Step: [ 500/ 600], Loss: 1.9025
Epoch: [ 1/ 5], Step: [ 600/ 600], Loss: 1.7986
Epoch: [ 2/ 5], Step: [ 100/ 600], Loss: 1.7351
Epoch: [ 2/ 5], Step: [ 200/ 600], Loss: 1.6751
Epoch: [ 2/ 5], Step: [ 300/ 600], Loss: 1.5865
Epoch: [ 2/ 5], Step: [ 400/ 600], Loss: 1.5560
Epoch: [ 2/ 5], Step: [ 500/ 600], Loss: 1.5834
Epoch: [ 2/ 5], Step: [ 600/ 600], Loss: 1.4815
Epoch: [ 3/ 5], Step: [ 100/ 600], Loss: 1.3608
Epoch: [ 3/ 5], Step: [ 200/ 600], Loss: 1.4468
Epoch: [ 3/ 5], Step: [ 300/ 600], Loss: 1.3153
Epoch: [ 3/ 5], Step: [ 400/ 600], Loss: 1.3212
Epoch: [ 3/ 5], Step: [ 500/ 600], Loss: 1.2410
Epoch: [ 3/ 5], Step: [ 600/ 600], Loss: 1.2144
Epoch: [ 4/ 5], Step: [ 100/ 600], Loss: 1.2006
Epoch: [ 4/ 5], Step: [ 200/ 600], Loss: 1.1102
Epoch: [ 4/ 5], Step: [ 300/ 600], Loss:

..\aten\src\ATen\native\BinaryOps.cpp:81: UserWarning: Integer division of tensors using div or / is deprecated, and in a future release div will perform true division as in Python 3. Use true_divide or floor_divide (// in Python) instead.


## **Q4-1)**

model을 학습하는 for 구문에 weight를 update하는 수식 **(w.data = w.data - lr * w.grad.item( ))** 과 다시 weight를 초기화하는 구문 **(w.grad.data.zero_( ))**을 넣어야 할 것이다.

## **Q4-2)**

**1. Adam**

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[1], [2], [3]])

class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1, 1)
        
    def forward(self, x):
        return self.linear(x)
    
# 모델 초기화
model = LinearRegressionModel()

# optimizer 설정
optimizer = optim.Adam(model.parameters(), lr=0.01)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    prediction = model(x_train)
    
    # cost 계산
    cost = F.mse_loss(prediction, y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad() # 미분값이
    cost.backward()
    optimizer.step()
    
    # 100번마다 로그 출력
    if epoch % 100 == 0:
        params = list(model.parameters())
        W = params[0].item()
        b = params[1].item()
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(epoch, nb_epochs, W, b, cost.item()))

Epoch    0/1000 W: -0.656, b: 0.350 Cost: 10.811549
Epoch  100/1000 W: 0.132, b: 1.116 Cost: 0.911273
Epoch  200/1000 W: 0.393, b: 1.274 Cost: 0.249786
Epoch  300/1000 W: 0.463, b: 1.188 Cost: 0.205655
Epoch  400/1000 W: 0.517, b: 1.068 Cost: 0.166252
Epoch  500/1000 W: 0.574, b: 0.944 Cost: 0.129773
Epoch  600/1000 W: 0.630, b: 0.820 Cost: 0.097939
Epoch  700/1000 W: 0.684, b: 0.700 Cost: 0.071501
Epoch  800/1000 W: 0.734, b: 0.588 Cost: 0.050499
Epoch  900/1000 W: 0.780, b: 0.486 Cost: 0.034494
Epoch 1000/1000 W: 0.822, b: 0.395 Cost: 0.022775


**2. RMSprop**

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[1], [2], [3]])

class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1, 1)
        
    def forward(self, x):
        return self.linear(x)
    
# 모델 초기화
model = LinearRegressionModel()

# optimizer 설정
optimizer = optim.RMSprop(model.parameters(), lr=0.01)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    prediction = model(x_train)
    
    # cost 계산
    cost = F.mse_loss(prediction, y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad() # 미분값이
    cost.backward()
    optimizer.step()
    
    # 100번마다 로그 출력
    if epoch % 100 == 0:
        params = list(model.parameters())
        W = params[0].item()
        b = params[1].item()
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(epoch, nb_epochs, W, b, cost.item()))

Epoch    0/1000 W: 0.727, b: -0.897 Cost: 3.128026
Epoch  100/1000 W: 1.140, b: -0.313 Cost: 0.014280
Epoch  200/1000 W: 1.078, b: -0.175 Cost: 0.004492
Epoch  300/1000 W: 1.030, b: -0.067 Cost: 0.000669
Epoch  400/1000 W: 1.006, b: -0.014 Cost: 0.000029
Epoch  500/1000 W: 1.000, b: -0.001 Cost: 0.000000
Epoch  600/1000 W: 1.000, b: -0.000 Cost: 0.000000
Epoch  700/1000 W: 0.974, b: -0.025 Cost: 0.004319
Epoch  800/1000 W: 0.997, b: -0.003 Cost: 0.000120
Epoch  900/1000 W: 1.000, b: -0.000 Cost: 0.000001
Epoch 1000/1000 W: 0.998, b: -0.002 Cost: 0.000027


**3. Adagrad**

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[1], [2], [3]])

class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1, 1)
        
    def forward(self, x):
        return self.linear(x)
    
# 모델 초기화
model = LinearRegressionModel()

# optimizer 설정
optimizer = optim.Adagrad(model.parameters(), lr=0.01)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    prediction = model(x_train)
    
    # cost 계산
    cost = F.mse_loss(prediction, y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad() # 미분값이
    cost.backward()
    optimizer.step()
    
    # 100번마다 로그 출력
    if epoch % 100 == 0:
        params = list(model.parameters())
        W = params[0].item()
        b = params[1].item()
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(epoch, nb_epochs, W, b, cost.item()))

Epoch    0/1000 W: -0.842, b: 0.294 Cost: 13.991162
Epoch  100/1000 W: -0.671, b: 0.465 Cost: 10.159396
Epoch  200/1000 W: -0.595, b: 0.541 Cost: 8.721702
Epoch  300/1000 W: -0.537, b: 0.597 Cost: 7.721966
Epoch  400/1000 W: -0.490, b: 0.644 Cost: 6.946383
Epoch  500/1000 W: -0.449, b: 0.684 Cost: 6.312260
Epoch  600/1000 W: -0.413, b: 0.720 Cost: 5.777288
Epoch  700/1000 W: -0.381, b: 0.752 Cost: 5.316406
Epoch  800/1000 W: -0.351, b: 0.781 Cost: 4.913328
Epoch  900/1000 W: -0.324, b: 0.807 Cost: 4.556783
Epoch 1000/1000 W: -0.298, b: 0.832 Cost: 4.238591
